# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
# In this notebook, I only have the "Better" Mommentum Strategy Algorithm code.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
import yfinance as yf
import time

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [1]:
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)[0]
    return [ticker.replace(".","-") for ticker in table['Symbol'].tolist()]

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [ ]:
def get_periodPrice_return(ticker, period):
    try:
        df = yf.download(ticker, period=period, progress=False, auto_adjust=True)
        if len(df) < 2:
            print(f"{ticker}: Insufficient data")
            return None

        close_prices = df['Close'].iloc[:, 0] 
        
        start_price = float(close_prices.iloc[0])
        end_price = float(close_prices.iloc[-1])

        return round((end_price / start_price - 1) * 100, 2)
    except Exception as e:
        print(f"{ticker}: error - {e}")
        return None

: 

## Parsing Our API 

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [ ]:
periods = ['1mo', '3mo', '6mo', '1y']

def get_stats(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        price = info.get('currentPrice')
        
        if price is None: 
            print(f"Skipping {ticker}: market price = None!")
            return None
            
        return_list = []
        for period in periods:
            period_return = get_periodPrice_return(ticker, period)
            if period_return is None:
                print(f"Skipping {ticker}: period return = None!")
                return None
            return_list.append(period_return)
        
        result = {
            'Ticker': ticker,
            'Stock Price': price,
            'Number of Shares to Buy': 'N/A'
        }
        
        for i, period in enumerate(periods):
            result[f'{period}-Price Return'] = return_list[i]
        
        result['HQM Score (mean)'] = 'N/A'
        return result
    except Exception as e:
        print(f'Error fetching {ticker}: {str(e)}')
        return None

print(get_stats('MMM'))

: 

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
ticker_list = get_sp500_tickers()

stocks = []
for ticker in ticker_list[0:60]:
    result = get_stats(ticker)
    if result:
        stocks.append(result)

: 

In [ ]:
hqm_columns = [
    'Ticker', 
    'Stock Price', 
    'Number of Shares to Buy', 
    '1mo-Price Return', 
    '1mo-Return Percentile', 
    '3mo-Price Return', 
    '3mo-Return Percentile', 
    '6mo-Price Return', 
    '6mo-Return Percentile', 
    '1y-Price Return', 
    '1y-Return Percentile', 
    'HQM Score (mean)'
]
hqm_df = pd.DataFrame(stocks, columns=hqm_columns)
hqm_df

: 

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [ ]:
# Done later..

: 

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [110]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Portfiolio size: ")
    try:
        float(portfolio_size)
    except ValueError:
        print("Enter a real number")

In [111]:
portfolio_input()
print(portfolio_size)

Portfiolio size:  100000000


100000000


In [112]:
position_size = float(portfolio_size) / len(hqm_df.index)
print(position_size)

for i in range(len(hqm_df)):
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Stock Price'])

hqm_df

1960784.3137254901


,Ticker,Stock Price,Number of Shares to Buy,1mo-Price Return,1mo-Return Percentile,3mo-Price Return,3mo-Return Percentile,6mo-Price Return,6mo-Return Percentile,1y-Price Return,1y-Return Percentile,HQM Score (mean)
0,MMM,151.27,12962,0.34,NaN,10.21,NaN,0.82,NaN,21.42,NaN,N/A
1,AOS,72.06,27210,11.43,NaN,11.81,NaN,1.91,NaN,-13.14,NaN,N/A
2,ABT,126.54,15495,-4.91,NaN,-1.87,NaN,-1.34,NaN,22.55,NaN,N/A
3,ABBV,190.28,10304,2.75,NaN,-0.22,NaN,9.50,NaN,6.44,NaN,N/A
4,ACN,283.15,6924,-4.37,NaN,-3.44,NaN,-23.74,NaN,-12.64,NaN,N/A
5,ADBE,370.74,5288,-3.69,NaN,0.58,NaN,-15.47,NaN,-31.65,NaN,N/A
6,AMD,166.47,11778,15.86,NaN,72.70,NaN,44.74,NaN,18.92,NaN,N/A
7,AES,13.80,142085,27.66,NaN,37.85,NaN,24.74,NaN,-16.88,NaN,N/A
8,AFL,103.37,18968,-0.52,NaN,-4.19,NaN,-3.35,NaN,11.26,NaN,N/A
9,A,120.18,16315,0.85,NaN,12.69,NaN,-20.05,NaN,-11.81,NaN,N/A


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [ ]:
for row in hqm_df.index:
    for period in periods:
        change_col = f'{period}-Price Return'
        percentile_col = f'{period}-Return Percentile'
        hqm_df.loc[row, percentile_col] = score(hqm_df[change_col], hqm_df.loc[row, change_col])        
hqm_df

,Ticker,Stock Price,Number of Shares to Buy,1mo-Price Return,1mo-Return Percentile,3mo-Price Return,3mo-Return Percentile,6mo-Price Return,6mo-Return Percentile,1y-Price Return,1y-Return Percentile,HQM Score (mean)
0,MMM,151.27,12962,0.34,27.450980,10.21,54.901961,0.82,45.098039,21.42,72.549020,N/A
1,AOS,72.06,27210,11.43,88.235294,11.81,64.705882,1.91,50.980392,-13.14,11.764706,N/A
2,ABT,126.54,15495,-4.91,5.882353,-1.87,13.725490,-1.34,35.294118,22.55,76.470588,N/A
3,ABBV,190.28,10304,2.75,47.058824,-0.22,19.607843,9.50,72.549020,6.44,45.098039,N/A
4,ACN,283.15,6924,-4.37,7.843137,-3.44,3.921569,-23.74,1.960784,-12.64,13.725490,N/A
5,ADBE,370.74,5288,-3.69,11.764706,0.58,21.568627,-15.47,11.764706,-31.65,3.921569,N/A
6,AMD,166.47,11778,15.86,96.078431,72.70,100.000000,44.74,98.039216,18.92,66.666667,N/A
7,AES,13.80,142085,27.66,98.039216,37.85,92.156863,24.74,96.078431,-16.88,7.843137,N/A
8,AFL,103.37,18968,-0.52,21.568627,-4.19,1.960784,-3.35,29.411765,11.26,50.980392,N/A
9,A,120.18,16315,0.85,31.372549,12.69,66.666667,-20.05,3.921569,-11.81,15.686275,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [132]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for period in periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{period}-Return Percentile'])
    hqm_df.loc[row, 'HQM Score (mean)'] = mean(momentum_percentiles)
hqm_df

,Ticker,Stock Price,Number of Shares to Buy,1mo-Price Return,1mo-Return Percentile,3mo-Price Return,3mo-Return Percentile,6mo-Price Return,6mo-Return Percentile,1y-Price Return,1y-Return Percentile,HQM Score (mean)
0,ANET,114.28,17157,12.49,90.196078,45.93,98.039216,13.94,86.274510,43.20,96.078431,92.647059
1,APH,105.02,18670,8.29,74.509804,38.65,94.117647,56.29,100.000000,65.97,100.000000,92.156863
2,AMD,166.47,11778,15.86,96.078431,72.70,100.000000,44.74,98.039216,18.92,66.666667,90.196078
3,ALLE,165.39,11855,15.34,94.117647,20.49,84.313725,22.56,94.117647,22.25,74.509804,86.764706
4,AES,13.80,142085,27.66,98.039216,37.85,92.156863,24.74,96.078431,-16.88,7.843137,73.529412
5,AZO,3820.91,513,8.48,78.431373,5.61,39.215686,13.29,84.313725,23.64,78.431373,70.098039
6,AMZN,231.44,8472,6.60,68.627451,23.30,86.274510,-1.69,33.333333,26.82,86.274510,68.627451
7,GOOGL,193.18,10150,11.32,86.274510,20.42,82.352941,0.95,47.058824,16.24,58.823529,68.627451
8,GOOG,194.08,10102,11.27,84.313725,19.64,80.392157,0.39,43.137255,15.61,54.901961,65.686275
9,APTV,70.91,27651,4.37,50.980392,26.00,90.196078,10.50,80.392157,3.16,39.215686,65.196078


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [133]:
hqm_df = hqm_df.sort_values('HQM Score (mean)', ascending=False, ignore_index=True)
hqm_df = hqm_df.head(50)
hqm_df

,Ticker,Stock Price,Number of Shares to Buy,1mo-Price Return,1mo-Return Percentile,3mo-Price Return,3mo-Return Percentile,6mo-Price Return,6mo-Return Percentile,1y-Price Return,1y-Return Percentile,HQM Score (mean)
0,ANET,114.28,17157,12.49,90.196078,45.93,98.039216,13.94,86.274510,43.20,96.078431,92.647059
1,APH,105.02,18670,8.29,74.509804,38.65,94.117647,56.29,100.000000,65.97,100.000000,92.156863
2,AMD,166.47,11778,15.86,96.078431,72.70,100.000000,44.74,98.039216,18.92,66.666667,90.196078
3,ALLE,165.39,11855,15.34,94.117647,20.49,84.313725,22.56,94.117647,22.25,74.509804,86.764706
4,AES,13.80,142085,27.66,98.039216,37.85,92.156863,24.74,96.078431,-16.88,7.843137,73.529412
5,AZO,3820.91,513,8.48,78.431373,5.61,39.215686,13.29,84.313725,23.64,78.431373,70.098039
6,AMZN,231.44,8472,6.60,68.627451,23.30,86.274510,-1.69,33.333333,26.82,86.274510,68.627451
7,GOOGL,193.18,10150,11.32,86.274510,20.42,82.352941,0.95,47.058824,16.24,58.823529,68.627451
8,GOOG,194.08,10102,11.27,84.313725,19.64,80.392157,0.39,43.137255,15.61,54.901961,65.686275
9,APTV,70.91,27651,4.37,50.980392,26.00,90.196078,10.50,80.392157,3.16,39.215686,65.196078


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [145]:
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Stock Price'])
hqm_df

,Ticker,Stock Price,Number of Shares to Buy,1mo-Price Return,1mo-Return Percentile,3mo-Price Return,3mo-Return Percentile,6mo-Price Return,6mo-Return Percentile,1y-Price Return,1y-Return Percentile,HQM Score (mean)
0,ANET,114.28,17157,12.49,90.196078,45.93,98.039216,13.94,86.274510,43.20,96.078431,92.647059
1,APH,105.02,18670,8.29,74.509804,38.65,94.117647,56.29,100.000000,65.97,100.000000,92.156863
2,AMD,166.47,11778,15.86,96.078431,72.70,100.000000,44.74,98.039216,18.92,66.666667,90.196078
3,ALLE,165.39,11855,15.34,94.117647,20.49,84.313725,22.56,94.117647,22.25,74.509804,86.764706
4,AES,13.80,142085,27.66,98.039216,37.85,92.156863,24.74,96.078431,-16.88,7.843137,73.529412
5,AZO,3820.91,513,8.48,78.431373,5.61,39.215686,13.29,84.313725,23.64,78.431373,70.098039
6,AMZN,231.44,8472,6.60,68.627451,23.30,86.274510,-1.69,33.333333,26.82,86.274510,68.627451
7,GOOGL,193.18,10150,11.32,86.274510,20.42,82.352941,0.95,47.058824,16.24,58.823529,68.627451
8,GOOG,194.08,10102,11.27,84.313725,19.64,80.392157,0.39,43.137255,15.61,54.901961,65.686275
9,APTV,70.91,27651,4.37,50.980392,26.00,90.196078,10.50,80.392157,3.16,39.215686,65.196078


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('mommentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='Mommentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [181]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0"%"',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

header_format = writer.book.add_format(
    {
    'bold': True,
    'align': 'center',
    'valign': 'vcenter',
    'border': 1
    }
)

In [182]:
column_formats = {
    'A' : ['Ticker', string_template],
    'B' : ['Stock Price', dollar_template],
    'C' : ['Number of Shares to Buy', integer_template],
    'D' : ['1mo-Price Return', percent_template],
    'E' : ['1mo-Return Percentile', percent_template],
    'F' : ['3mo-Price Return', percent_template],
    'G' : ['3mo-Return Percentile', percent_template],
    'H' : ['6mo-Price Return', percent_template],
    'I' : ['6mo-Return Percentile', percent_template],
    'J' : ['1y-Price Return', percent_template],
    'K' : ['1y-Return Percentile', percent_template],
    'L' : ['HQM Score (mean)', percent_template]
}

In [183]:
worksheet = writer.sheets['Mommentum Strategy']
for column, (header, cell_format) in column_formats.items():
    worksheet.set_column(
        f'{column}:{column}', 
        25, 
        cell_format
    )
    worksheet.write(f'{column}1', header, header_format)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [184]:
writer.close()